# Data Science Intern @Lets Grow More

# Author : Iqra Ayaz 

# Task 8 : Next Word Prediction

In [5]:
conda install tensorflow

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... done

## Package Plan ##

  environment location: C:\Users\Ahmed\anaconda3

  added / updated specs:
    - tensorflow


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _tflow_select-2.3.0        |              mkl           3 KB
    abseil-cpp-20211102.0      |       hd77b12b_0         1.7 MB
    absl-py-1.4.0              |  py310haa95532_0         181 KB
    aiohttp-3.8.3              |  py310h2bbff1b_0         418 KB
    aiosignal-1.2.0            |     pyhd3eb1b0_0          12 KB
    astunparse-1.6.3           |             py_0          17 KB
    async-timeout-4.0.2



==> WARNING: A newer version of conda exists. <==
  current version: 23.3.1
  latest version: 23.7.2

Please update conda by running

    $ conda update -n base -c defaults conda

Or to minimize the number of packages updated during conda update use

     conda install conda=23.7.2




tensorflow-base-2.10 | 76.1 MB   |            |   0% 















grpcio-1.48.2        | 2.0 MB    | ##2        |  23% 
















tensorflow-base-2.10 | 76.1 MB   |            |   0% 















grpcio-1.48.2        | 2.0 MB    | ##4        |  25% 
















tensorflow-base-2.10 | 76.1 MB   |            |   0% 















grpcio-1.48.2        | 2.0 MB    | ##6        |  26% 
















tensorflow-base-2.10 | 76.1 MB   |            |   0% 















grpcio-1.48.2        | 2.0 MB    | ##7        |  28% 
















tensorflow-base-2.10 | 76.1 MB   |            |   1% 















grpcio-1.48.2        | 2.0 MB    | ##9        |  29% 
















tensorflow-base-2.10 | 76.1 MB   |            |   1% 
















tensorflow-base-2.10 | 76.1 MB   |            |   1% 















grpcio-1.48.2        | 2.0 MB    | ###        |  31% 
















tensorflow-base-2.10 | 76.1 MB   |            |   1% 















grpcio-1.48.2        | 2.0 

In [3]:
import numpy as np
import pickle
import heapq
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from tensorflow.keras.optimizers import RMSprop
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
text = open('sample.txt',encoding='UTF-8').read().lower()
tr = RegexpTokenizer(r'\w+')
words = tr.tokenize(text)
unique_words = np.unique(words)
uw_index = dict((c, i) for i, c in enumerate(unique_words))

In [ ]:
len_w = 5
prev = []
nextw = []
for i in range(len(words) - len_w):
    prev.append(words[i:i + len_w])
    nextw.append(words[i + len_w])
print(prev[0],nextw[0])

In [ ]:
X = np.zeros((len(prev), len_w, len(unique_words)), dtype=bool)
Y = np.zeros((len(nextw), len(unique_words)), dtype=bool)

In [ ]:
for i, each_words in enumerate(prev):
    for j, each_word in enumerate(each_words):
        X[i, j, uw_index[each_word]] = 1
    Y[i, uw_index[nextw[i]]] = 1

In [ ]:
print(X[0][1])

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(len_w, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=5, shuffle=True).history

In [ ]:
model.save('word-pred-model.h5')
pickle.dump(history, open("history.p", "wb"))
model = load_model('word-pred-model.h5')
history = pickle.load(open("history.p", "rb"))

plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epochs')
plt.legend(['train', 'test'], loc='lower right')

In [ ]:
def prepare_input(text):
    x = np.zeros((1, len_w, len(unique_words)))
    for t, word in enumerate(text.split()):
        print(word)
        x[0, t, uw_index[word]] = 1
    return x

def sample(preds, top_n=3):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    return heapq.nlargest(top_n, range(len(preds)), preds.take)

def predict_completion(text):
    original_text = text
    generated = text
    completion = ''
    while True:
        x = prepare_input(text)
        preds = model.predict(x, verbose=0)[0]
        next_index = sample(preds, top_n=1)[0]
        next_char = indices_char[next_index]
        text = text[1:] + next_char
        completion += next_char
        if len(original_text + completion) + 2 > len(original_text) and next_char == ' ':
            return completion
        
def predict_word(text, n=3):
    x = prepare_input(text)
    preds = model.predict(x, verbose=0)[0]
    next_indices = sample(preds, n)
    return [unique_words[idx] for idx in next_indices]

In [ ]:
quotes = [
    "It is not a lack of love, but a lack of friendship that makes unhappy marriages.",
    "That which does not kill us makes us stronger.",
    "I'm not upset that you lied to me, I'm upset that from now on I can't believe you.",
    "And those who were seen dancing were thought to be insane by those who could not hear the music.",
    "It is hard enough to remember my opinions, without also remembering my reasons for them!"
]

for q in quotes:
    print("original sentence: ",q)
    seq = " ".join(tr.tokenize(q.lower())[0:5])
    print("sequence: ",seq)
    print("next possible words: ", predict_word(seq, 5))